In [34]:
import numpy as np
import pandas as pd
import os
import cv2
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
#
import scienceplots
from cmcrameri import cm

In [4]:
plt.style.use(['science', 'grid'])

In [8]:
df = pd.read_csv("data.csv")
# Ensure we are testing diffraction pattern data
df = df.loc[df.seed.str.match('^pattern.*')]

groups = sorted(df.group.unique())
space_groups = pd.read_csv("space_groups.csv", index_col="ICSD_code").to_dict()["space_group"]

In [32]:
dataset_dir = "../datasets/FDP"
results_dir = "../results"

differences = {group: np.zeros((128, 128)) for group in groups}
differences_counts = {group: 0 for group in groups}

for index, row in df.iterrows():
    code = row["code"]
    group = row["group"]
    seed = row["seed"]
    phase = row["phase"]
    epoch = row["epoch"]
    
    if (phase == "test" or phase == "val") and epoch == 100:
        fake = cv2.cvtColor(cv2.imread(
            os.path.join(results_dir, seed, phase + "_100", "images", str(code) + "_structure_synthesized_image.png")
        ), cv2.COLOR_BGR2GRAY) / 255
        real = cv2.cvtColor(cv2.imread(
            os.path.join(dataset_dir, str(code), str(code) + "_+0+0+0.png")
        ), cv2.COLOR_BGR2GRAY) / 255
        
        diff = (real - fake)**2
        
        differences[group] += diff
        differences_counts[group] += 1

for group in groups:
    differences[group] /= differences_counts[group]

In [41]:
cmap = cm.managua_r

In [56]:
results_dirs = [os.path.join("../results", d) for d in os.listdir("../results") if re.match('^pattern.*', d)]
dataset_dir = "../datasets/FDP"

for group in groups:
    image = differences[group]
    image_colour = cmap(image)
    image_colour
    output = os.path.join("figures", "group differences")
    Path(output).mkdir(parents=True, exist_ok=True)

    cv2.imwrite(os.path.join(output, group + ".png"), image_colour * 255)